In [30]:
import numpy as np
import itertools
from itertools import cycle, islice
import operator

In [4]:
def minimize(expr):
    if '+' not in expr or '*' not in expr:
        return eval(expr)
    if len(expr)==1:
        if expr.isdigit():
            return int(expr)
        else:
            return "Invalid symbol"
    res = np.inf
    for i in range(len(expr) - 1):
        if expr[i] == '+':
            left = minimize(expr[:i])
            right = minimize(expr[i + 1 :len(expr)])
            res = min(res, left + right)
        if expr[i] == '*':
            left = minimize(expr[:i])
            right = minimize(expr[i + 1 :len(expr)])
            res = min(res, left * right)
    return res

In [5]:
minimize("1*1+1")

2

In [6]:
#only * and +
def maximize (expr):
    if '+' not in expr or '*' not in expr:
        return eval(expr)
    if len(expr)==1:
        if expr.isdigit():
            return int(expr)
        else:
            return "Invalid symbol"
    res = -1
    for i in range(len(expr) - 1):
        if expr[i] == '+':
            left = maximize(expr[:i])
            #print("left", expr[:i])
            #print("right", expr[i + 1 :len(expr)])
            right = maximize(expr[i + 1 :len(expr)])
            res = max(res, left + right)
        if expr[i] == '*':
            #print("left", expr[:i])
            #print("right", expr[i + 1 :len(expr)])
            left = maximize(expr[:i])
            right = maximize(expr[i + 1 :len(expr)])
            res = max(res, left * right)
    return res

In [7]:
maximize("1+1*1*2")

4

In [8]:
def put_operators(numbers, operators):
    results = {}
    symbols = numbers+operators
    permutations = list(itertools.permutations(symbols))
    for perm in permutations:
        str_perm = ''.join(str(e) for e in perm)
        try:
            e = eval(str_perm)
            results[str_perm] = maximize(str_perm)
        except:
            continue
    max_key = max(results, key=lambda k: results[k])
    return max_key, results[max_key]

In [9]:
put_operators([1, 2, 3], ['*', '*'])

('3**21', 10460353203)

works only with given operators, ** considered by eval as normal - degree

In [10]:
def put_operators2(numbers, operators):
    oper_nr = len(numbers)-1
    opers = list(itertools.product(operators, repeat=oper_nr))
    return opers

In [11]:
put_operators2([1, 2, 3, 4], ['*', '+'])

[('*', '*', '*'),
 ('*', '*', '+'),
 ('*', '+', '*'),
 ('*', '+', '+'),
 ('+', '*', '*'),
 ('+', '*', '+'),
 ('+', '+', '*'),
 ('+', '+', '+')]

In [27]:
def roundrobin(*iterables):
    "roundrobin('ABC', 'D', 'EF') --> A D E B F C"
    # Recipe credited to George Sakkis
    num_active = len(iterables)
    nexts = cycle(iter(it).__next__ for it in iterables)
    while num_active:
        try:
            for next in nexts:
                yield next()
        except StopIteration:
            # Remove the iterator we just exhausted from the cycle.
            num_active -= 1
            nexts = cycle(islice(nexts, num_active))

In [40]:
def put_operators_final(numbers, operators):
    results={}
    opers = put_operators2(numbers, operators)
    for oper in opers:
        str_expr = ''.join(str(e) for e in roundrobin(numbers, oper))
        results[str_expr] = maximize(str_expr)
    max_key = max(results, key=lambda k: results[k])
    print(results)
    return max_key, results[max_key]

In [43]:
put_operators_final([1, 2, 3, 4, 5], ['*', '+'])

{'1*2*3*4*5': 120, '1*2*3*4+5': 54, '1*2*3+4*5': 70, '1*2*3+4+5': 24, '1*2+3*4*5': 100, '1*2+3*4+5': 45, '1*2+3+4*5': 45, '1*2+3+4+5': 14, '1+2*3*4*5': 180, '1+2*3*4+5': 81, '1+2*3+4*5': 105, '1+2*3+4+5': 36, '1+2+3*4*5': 120, '1+2+3*4+5': 54, '1+2+3+4*5': 50, '1+2+3+4+5': 15}


('1+2*3*4*5', 180)